In [1]:
import numpy as np
import pandas as pd
import re
import collections
import operator
import simpy
import random

In [2]:
trasacao = pd.read_csv("data/POS_transactions_20190328-20190410.csv")

In [3]:
trasacao.head()

,WorkstationGroupID,TranID,BeginDateTime,EndDateTime,OperatorID,TranTime,BreakTime,ArtNum,TNcash,TNcard,Amount
0,1,1.904031e+13,2019-04-03T13:28:17,2019-04-03T13:28:17,10,0,1,1,False,False,11.99
1,1,1.903291e+13,2019-03-29T14:04:50,2019-03-29T14:05:31,101,41,21,6,True,False,13.74
2,1,1.903291e+13,2019-03-29T14:05:52,2019-03-29T14:06:35,101,43,44,12,True,False,97.47
3,1,1.903291e+13,2019-03-29T14:07:19,2019-03-29T14:07:43,101,24,25,6,False,True,37.18
4,1,1.903291e+13,2019-03-29T14:08:08,2019-03-29T14:09:44,101,96,18,48,True,False,154.70


In [4]:
trasacao.dtypes

WorkstationGroupID      int64
TranID                float64
BeginDateTime          object
EndDateTime            object
OperatorID              int64
TranTime                int64
BreakTime               int64
ArtNum                  int64
TNcash                   bool
TNcard                   bool
Amount                float64
dtype: object

In [5]:
trasacao_workstation_service = trasacao.query('WorkstationGroupID == 1')

In [6]:
trasacao_workstation_service_date = trasacao_workstation_service.loc[trasacao_workstation_service.BeginDateTime.str.contains('2019-04-03T10|2019-04-03T11|2019-04-03T12', flags=re.I, regex=True)]

## Estatistica descritiva campo TranTime

In [7]:
print("media",trasacao_workstation_service_date["TranTime"].mean())
print("mediana",trasacao_workstation_service_date["TranTime"].median())
print("Moda",list(trasacao_workstation_service_date["TranTime"].mode()))
print("minimo",trasacao_workstation_service_date["TranTime"].min())
print("maximo",trasacao_workstation_service_date["TranTime"].max())
print("amplitude",trasacao_workstation_service_date["TranTime"].max()-trasacao_workstation_service_date["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date["TranTime"].std())
print("variacia",trasacao_workstation_service_date["TranTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date["TranTime"].std()/trasacao_workstation_service_date["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date["TranTime"].skew())

media 50.55750487329435
mediana 41.0
Moda [26, 31]
minimo 0
maximo 309
amplitude 309
desvio padrão 40.12457538731323
variacia 1609.9815500121827
coeficinte de variação 0.7936423185414746
Assimetria 2.0333015820729505


## Estatistica descritiva campo Breaktime

In [8]:
print("media",trasacao_workstation_service_date["BreakTime"].mean())
print("mediana",trasacao_workstation_service_date["BreakTime"].median())
print("Moda",list(trasacao_workstation_service_date["BreakTime"].mode()))
print("minimo",trasacao_workstation_service_date["BreakTime"].min())
print("maximo",trasacao_workstation_service_date["BreakTime"].max())
print("amplitude",trasacao_workstation_service_date["BreakTime"].max()-trasacao_workstation_service_date["BreakTime"].min())
print("desvio padrão",trasacao_workstation_service_date["BreakTime"].std())
print("variacia",trasacao_workstation_service_date["BreakTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date["BreakTime"].std()/trasacao_workstation_service_date["BreakTime"].mean())
print("Assimetria",trasacao_workstation_service_date["BreakTime"].skew())


media 33.44054580896686
mediana 21.0
Moda [17]
minimo 6
maximo 1147
amplitude 1141
desvio padrão 71.21608892361684
variacia 5071.731321576502
coeficinte de variação 2.129632971018096
Assimetria 12.022677107628233


## Estatistica Descritiva campo numeros de artigos

In [9]:
print("media",trasacao_workstation_service_date["ArtNum"].mean())
print("mediana",trasacao_workstation_service_date["ArtNum"].median())
print("Moda",list(trasacao_workstation_service_date["ArtNum"].mode()))
print("minimo",trasacao_workstation_service_date["ArtNum"].min())
print("maximo",trasacao_workstation_service_date["ArtNum"].max())
print("amplitude",trasacao_workstation_service_date["ArtNum"].max()-trasacao_workstation_service_date["ArtNum"].min())
print("desvio padrão",trasacao_workstation_service_date["ArtNum"].std())
print("variacia",trasacao_workstation_service_date["ArtNum"].var())
print("coeficinte de variação",trasacao_workstation_service_date["ArtNum"].std()/trasacao_workstation_service_date["ArtNum"].mean())
print("Assimetria",trasacao_workstation_service_date["ArtNum"].skew())

media 15.19103313840156
mediana 11.0
Moda [3]
minimo 1
maximo 123
amplitude 122
desvio padrão 14.937191337843698
variacia 223.1196850633528
coeficinte de variação 0.9832900239078426
Assimetria 2.6803168909075863


## Estatistica Descritiva campo TNcash e TNcard

In [10]:
trasacao_workstation_service_date["TNcash"].describe()

count      513
unique       2
top       True
freq       335
Name: TNcash, dtype: object

## Calculando o tempo medio de item por segundo



In [11]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)
media_produtos_segundos

9.170826263339407

##  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [12]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    if trasacao["TranTime"]>0:
        produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)
media_produtos_segundos

4.642332924547316

##  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [13]:

produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)
media_produtos_segundos

4.528493338792098

## Estatistica descritiva considerando apenas TNcash


In [14]:
trasacao_workstation_service_date_tncash = trasacao_workstation_service_date.query('TNcash == True')

### Estatistica descritiva campo TranTime

In [15]:
print("media",trasacao_workstation_service_date_tncash["TranTime"].mean())
print("mediana",trasacao_workstation_service_date_tncash["TranTime"].median())
print("Moda",list(trasacao_workstation_service_date_tncash["TranTime"].mode()))
print("minimo",trasacao_workstation_service_date_tncash["TranTime"].min())
print("maximo",trasacao_workstation_service_date_tncash["TranTime"].max())
print("amplitude",trasacao_workstation_service_date_tncash["TranTime"].max()-trasacao_workstation_service_date_tncash["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date_tncash["TranTime"].std())
print("variacia",trasacao_workstation_service_date_tncash["TranTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncash["TranTime"].std()/trasacao_workstation_service_date_tncash["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date_tncash["TranTime"].skew())

media 44.208955223880594
mediana 35.0
Moda [11, 26, 28]
minimo 4
maximo 309
amplitude 305
desvio padrão 37.53096019896204
variacia 1408.5729734560728
coeficinte de variação 0.8489447445410051
Assimetria 2.6301824423971367


### Estatistica descritiva campo BreakTime

In [16]:
print("media",trasacao_workstation_service_date_tncash["BreakTime"].mean())
print("mediana",trasacao_workstation_service_date_tncash["BreakTime"].median())
print("Moda",list(trasacao_workstation_service_date_tncash["BreakTime"].mode()))
print("minimo",trasacao_workstation_service_date_tncash["BreakTime"].min())
print("maximo",trasacao_workstation_service_date_tncash["BreakTime"].max())
print("amplitude",trasacao_workstation_service_date_tncash["BreakTime"].max()-trasacao_workstation_service_date_tncash["BreakTime"].min())
print("desvio padrão",trasacao_workstation_service_date_tncash["BreakTime"].std())
print("variacia",trasacao_workstation_service_date_tncash["BreakTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncash["BreakTime"].std()/trasacao_workstation_service_date_tncash["BreakTime"].mean())
print("Assimetria",trasacao_workstation_service_date_tncash["BreakTime"].skew())

media 33.56417910447761
mediana 22.0
Moda [17, 19]
minimo 8
maximo 940
amplitude 932
desvio padrão 57.94443885705379
variacia 3357.5579944588453
coeficinte de variação 1.7263773583344912
Assimetria 12.235668470386164


### Estatistica descritiva campo ArtNum

In [17]:
print("media",trasacao_workstation_service_date_tncash["ArtNum"].mean())
print("mediana",trasacao_workstation_service_date_tncash["ArtNum"].median())
print("Moda",list(trasacao_workstation_service_date_tncash["ArtNum"].mode()))
print("minimo",trasacao_workstation_service_date_tncash["ArtNum"].min())
print("maximo",trasacao_workstation_service_date_tncash["ArtNum"].max())
print("amplitude",trasacao_workstation_service_date_tncash["ArtNum"].max()-trasacao_workstation_service_date_tncash["ArtNum"].min())
print("desvio padrão",trasacao_workstation_service_date_tncash["ArtNum"].std())
print("variacia",trasacao_workstation_service_date_tncash["ArtNum"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncash["ArtNum"].std()/trasacao_workstation_service_date_tncash["ArtNum"].mean())
print("Assimetria",trasacao_workstation_service_date_tncash["ArtNum"].skew())

media 13.170149253731344
mediana 10.0
Moda [3]
minimo 1
maximo 123
amplitude 122
desvio padrão 13.458663871692076
variacia 181.13563321118957
coeficinte de variação 1.0219067082993756
Assimetria 3.526173671756291


### Calculando o tempo medio de item por segundo

In [18]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncash.iterrows():
    produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncash.index)
media_produtos_segundos

9.980679204243227

###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [19]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncash.iterrows():
    if trasacao["TranTime"]>0:
        produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncash.index)
media_produtos_segundos

4.537310756315868

###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [20]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncash.iterrows():
    produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncash.index)
media_produtos_segundos

5.4433684479273605

## Estatistica descritiva considerando apenas TNcard

In [21]:
trasacao_workstation_service_date_tncard = trasacao_workstation_service_date.query('TNcard == True')

### Estatistica descritiva campo TranTime

In [22]:
print("media",trasacao_workstation_service_date_tncard["TranTime"].mean())
print("mediana",trasacao_workstation_service_date_tncard["TranTime"].median())
print("Moda",list(trasacao_workstation_service_date_tncard["TranTime"].mode()))
print("minimo",trasacao_workstation_service_date_tncard["TranTime"].min())
print("maximo",trasacao_workstation_service_date_tncard["TranTime"].max())
print("amplitude",trasacao_workstation_service_date_tncard["TranTime"].max()-trasacao_workstation_service_date_tncard["TranTime"].min())
print("desvio padrão",trasacao_workstation_service_date_tncard["TranTime"].std())
print("variacia",trasacao_workstation_service_date_tncard["TranTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncard["TranTime"].std()/trasacao_workstation_service_date_tncard["TranTime"].mean())
print("Assimetria",trasacao_workstation_service_date_tncard["TranTime"].skew())

media 64.33714285714285
mediana 52.0
Moda [31]
minimo 13
maximo 251
amplitude 238
desvio padrão 41.99487575277451
variacia 1763.5695894909688
coeficinte de variação 0.6527314376708002
Assimetria 1.3877977720260803


### Estatistica descritiva campo BreakTime

In [23]:
print("media",trasacao_workstation_service_date_tncard["BreakTime"].mean())
print("mediana",trasacao_workstation_service_date_tncard["BreakTime"].median())
print("Moda",list(trasacao_workstation_service_date_tncard["BreakTime"].mode()))
print("minimo",trasacao_workstation_service_date_tncard["BreakTime"].min())
print("maximo",trasacao_workstation_service_date_tncard["BreakTime"].max())
print("amplitude",trasacao_workstation_service_date_tncard["BreakTime"].max()-trasacao_workstation_service_date_tncard["BreakTime"].min())
print("desvio padrão",trasacao_workstation_service_date_tncard["BreakTime"].std())
print("variacia",trasacao_workstation_service_date_tncard["BreakTime"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncard["BreakTime"].std()/trasacao_workstation_service_date_tncard["BreakTime"].mean())
print("Assimetria",trasacao_workstation_service_date_tncard["BreakTime"].skew())

media 32.48
mediana 16.0
Moda [14]
minimo 6
maximo 1147
amplitude 1141
desvio padrão 91.86445787511983
variacia 8439.078620689665
coeficinte de variação 2.828339220293098
Assimetria 10.706897789257468


### Estatistica descritiva campo ArtNum

In [24]:
print("media",trasacao_workstation_service_date_tncard["ArtNum"].mean())
print("mediana",trasacao_workstation_service_date_tncard["ArtNum"].median())
print("Moda",list(trasacao_workstation_service_date_tncard["ArtNum"].mode()))
print("minimo",trasacao_workstation_service_date_tncard["ArtNum"].min())
print("maximo",trasacao_workstation_service_date_tncard["ArtNum"].max())
print("amplitude",trasacao_workstation_service_date_tncard["ArtNum"].max()-trasacao_workstation_service_date_tncard["ArtNum"].min())
print("desvio padrão",trasacao_workstation_service_date_tncard["ArtNum"].std())
print("variacia",trasacao_workstation_service_date_tncard["ArtNum"].var())
print("coeficinte de variação",trasacao_workstation_service_date_tncard["ArtNum"].std()/trasacao_workstation_service_date_tncard["ArtNum"].mean())
print("Assimetria",trasacao_workstation_service_date_tncard["ArtNum"].skew())

media 18.925714285714285
mediana 13.0
Moda [3]
minimo 1
maximo 98
amplitude 97
desvio padrão 17.016225233061128
variacia 289.5519211822662
coeficinte de variação 0.8991061038000294
Assimetria 1.7948384626166136


### Calculando o tempo medio de item por segundo

In [25]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncard.iterrows():
    produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncard.index)
media_produtos_segundos

7.877840225663656

### Calculando o tempo medio de item por transação

In [26]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncard.iterrows():
    produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncard.index)
media_produtos_segundos

5.0663667686371365

### Calculando o tempo medio item por tempo de empacotamento

In [27]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date_tncard.iterrows():
    produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date_tncard.index)
media_produtos_segundos

2.8114734570265214

In [28]:
frequencia = collections.Counter(trasacao_workstation_service_date["ArtNum"])

In [29]:
frequencia_ordenada = sorted(frequencia.items(), key=operator.itemgetter(1), reverse=True)

In [30]:
frequencia_porcentagem = {}

In [31]:
frequencia_f = []
for frequencia in frequencia_ordenada:
    porcentagem = (frequencia[1]/len(trasacao_workstation_service_date.index))*100
    frequencia_f.append((frequencia[0],porcentagem))


In [32]:
frequencia_dividida = {'a':[1,23,1]}
frequencia_dividida['a'].append(4)


In [33]:
frequencia_dividida_tamanho = {"qtd_pequena": [], "qtd_media": [], "qtd_grande": []}
soma_frequencia_pequena = 0
soma_frequencia_media = 0
soma_frequencia_grande = 0
for frequencia in frequencia_f:
    if frequencia[0] <= 10 :
        frequencia_dividida_tamanho["qtd_pequena"].append(frequencia[0])
        soma_frequencia_pequena += frequencia[1] 
    elif (frequencia[0] > 10) & (frequencia[0]<=30):
        frequencia_dividida_tamanho["qtd_media"].append(frequencia[0])
        soma_frequencia_media += frequencia[1]
    else :
        frequencia_dividida_tamanho["qtd_grande"].append(frequencia[0])
        soma_frequencia_grande += frequencia[1]
frequencia_dividida_tamanho["qtd_pequena"] = (soma_frequencia_pequena)
frequencia_dividida_tamanho["qtd_media"] = (soma_frequencia_media)
frequencia_dividida_tamanho["qtd_grande"] = (frequencia_dividida_tamanho["qtd_grande"],soma_frequencia_grande)

In [34]:
frequencia_dividida_tamanho["qtd_pequena"]

48.927875243664715

In [35]:
frequencia_dividida_tamanho["qtd_media"]

39.96101364522416

In [36]:
frequencia_dividida_tamanho["qtd_grande"]

([37,
  42,
  31,
  40,
  48,
  33,
  41,
  44,
  39,
  35,
  34,
  38,
  36,
  47,
  62,
  58,
  67,
  59,
  32,
  98,
  82,
  45,
  123,
  53,
  87,
  49,
  76,
  51,
  46,
  64,
  70,
  72,
  84],
 11.111111111111118)

## Calculando o tempo medio de item por segundo

In [37]:

produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)
media_produtos_segundos

9.170826263339407

In [38]:

produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)
media_produtos_segundos

4.528493338792098

In [39]:
produtos_segundos = 0
for index, trasacao in trasacao_workstation_service_date.iterrows():
    if trasacao["TranTime"]>0:
        produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
media_produtos_segundos=produtos_segundos/len(trasacao_workstation_service_date.index)

In [40]:
media_produtos_segundos

4.642332924547316

In [41]:
OperatorIDs=trasacao_workstation_service_date[['OperatorID']]

In [42]:
operadores_unicos = OperatorIDs.drop_duplicates()
operadores_list = operadores_unicos.values

In [43]:
operadores = {}
for operador in operadores_list:
    operadores[int(operador)] = trasacao_workstation_service_date.query(f"OperatorID=={operador}")


In [44]:
operadores.keys()

dict_keys([105, 121, 128, 131, 301, 311, 317, 361])